In [2]:
import pandas as pd
import re
from transformers import AutoTokenizer, GPT2LMHeadModel

In [ ]:
INFER_MODEL_PATH = "./output"
BASE_MODEL = "skt/kogpt2-base-v2"

In [ ]:
infer_tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    max_len = 512,
    padding = "max_length",
    add_special_tokens = True,
    return_tensors = "pt",
    truncation = True,
    bos_token = "<s>",
    eos_token = "</s>",
    unk_token = "<unk>",
    pad_token = "<pad>",
    mask_token = "<mask>",
    sep_token = "<sep>"
)

In [ ]:
infer_model = GPT2LMHeadModel.from_pretrained(INFER_MODEL_PATH)
infer_model.eval()

In [5]:
data = pd.read_csv("../../data/raw_csv_data/600100001_data.csv")
title = data["title"][18]
description = data["description"][18]
title = re.sub(r"[^\w\s]", "", title)
description = description.replace("\n", " ")
description = description.replace("\r", " ")
description = re.sub(r"[^\w\s]", "", description)
text = "<s>" + title + "<sep>" + description 

In [ ]:
input_ids = infer_tokenizer.encode(
    text,
    add_special_tokens = True,
    return_tensors="pt"
)

output_sequence = infer_model.generate(
    input_ids = input_ids,
    do_sample=True,
    max_length = 512,
    num_return_sequences = 10,
    top_p=0.95,
    top_k=5
)

decode_output = infer_tokenizer.decode(output_sequence[0], skip_special_tokens=True)
decode_output = decode_output[len(text)+1:]
decode_output = decode_output.split()
decode_output = list(set(decode_output))
print(decode_output)

In [ ]:
print(text)